In [13]:
import json
import pandas as pd

In [14]:
files = ["/Users/elhidze/Gagarin/metadata/train/normal/0.json",
        "/Users/elhidze/Gagarin/metadata/train/normal/1.json",
        "/Users/elhidze/Gagarin/metadata/train/normal/2.json",
        "/Users/elhidze/Gagarin/metadata/train/normal/3.json",
        "/Users/elhidze/Gagarin/metadata/train/normal/4.json",
        "/Users/elhidze/Gagarin/metadata/train/normal/5.json",
        "/Users/elhidze/Gagarin/metadata/train/normal/6.json",
        "/Users/elhidze/Gagarin/metadata/train/normal/7.json",
        "/Users/elhidze/Gagarin/metadata/train/normal/8.json"]
outs = ['/Users/elhidze/Gagarin/meta_csv/train/normal/0.csv',
        '/Users/elhidze/Gagarin/meta_csv/train/normal/1.csv',
        '/Users/elhidze/Gagarin/meta_csv/train/normal/2.csv',
        '/Users/elhidze/Gagarin/meta_csv/train/normal/3.csv',
        '/Users/elhidze/Gagarin/meta_csv/train/normal/4.csv',
        '/Users/elhidze/Gagarin/meta_csv/train/normal/5.csv',
        '/Users/elhidze/Gagarin/meta_csv/train/normal/6.csv',
        '/Users/elhidze/Gagarin/meta_csv/train/normal/7.csv',
        '/Users/elhidze/Gagarin/meta_csv/train/normal/8.csv']

In [15]:
for i in range(len(files)):
    with open(files[i]) as f:
        frames = json.load(f)
    
    frames_json = json.dumps(frames)
    frames = json.loads(frames_json)
    
    pkt_pos = [int(frame['pkt_pos']) for frame in frames]
    byte_pos = pkt_pos
    time_pos = [i / 25 for i in range(len(pkt_pos))]
    pkt_sizes = [int(frame['pkt_size']) for frame in frames]
    m = max(pkt_sizes)
    pkt_sizes = [size / m for size in pkt_sizes]
    pic_types = [frame['pict_type'] for frame in frames]
    
    df = pd.DataFrame({'time_pos': time_pos, 'byte_pos': byte_pos, 'size': pkt_sizes, 'type': pic_types}).sort_values(by='time_pos')
    df.to_csv(outs[i], index=False)

In [16]:
anomaly_data = {
    '0': {
        'move': [(48, 52), (93, 98), (140, 142), (180, 185), (248, 249), (313, 315), (413, 414), (471, 474), (521, 522), (615, 619), (670, 674)]
    },
    '2': {
        'blur': [(19, 42), (81, 97), (100, 127), (186, 216), (301, 303), (372, 409), (525, 556), (828, 878), (983, 985), (1014, 1017), (1085, 1087), (1153, 1166)],
        'move': [(141, 142), (240, 247), (751, 756), (758, 762), (919, 924), (983, 985), (1014, 1017), (1085, 1087), (1141, 1142)]
    },
    '3': {
        'blur': [(19, 59), (79, 138)],
        'move': [(0, 19), (40, 60), (80, 106), (121, 141)]
    },
    '4': {
        'block': [(17, 28), (55, 64), (174, 185)],
        'overexposure': [(28, 30), (64, 70)],
        'blur': [(83, 153)]
    },
    '5': {
        'block': [(662, 675), (679, 690)],
        'move': [(3, 4), (324, 325), (454, 455), (507, 508), (508, 560), (547, 662), (714, 718)],
        'blur': [(161, 221)]
    }
}

count = 0
for table in outs:
    df = pd.read_csv(table)
    
    df['normal'] = 1
    df['has_blur'] = 0
    df['has_move'] = 0
    df['has_overexposure'] = 0
    df['has_block'] = 0
    
    # for anomaly_type, time_ranges in anomaly_data[str(count)].items():
    #     for start, end in time_ranges:
    #         df.loc[(df['time_pos'] >= start) & (df['time_pos'] <= end), f'has_{anomaly_type}'] = 1
    # 
    # df.loc[(df['has_blur'] == 0) & (df['has_move'] == 0) & (df['has_overexposure'] == 0) & (df['has_block'] == 0), 'normal'] = 1
    # df.loc[(df['has_blur'] == 1) | (df['has_move'] == 1) | (df['has_overexposure'] == 1) | (df['has_block'] == 1), 'normal'] = 0
    # 
    df.to_csv(table, index=False)
    
    if count == 0:
        count += 1
    count += 1